# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
os.chdir('..')
sys.path.append('src')

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from pathlib import Path
from datetime import datetime
import infer
import postprocessing
import utils
import numpy as np
import torch

# Code

In [ ]:
src_folder = '/mnt/storage/HuBMAP/test'
img_name = next(Path(src_folder).glob('*.tiff'))
print(img_name)

In [ ]:
p = '../2021_Feb_15_18_55_28_PAMBUH/'
foo = infer.get_infer_func(p)

In [ ]:
def test_infer(batch):
    tensor = torch.CharTensor if torch.cuda.is_available() else torch.cuda.CharTensor
    return torch.cat([tensor(i)[:1].unsqueeze(0) for i in batch], 0)

In [ ]:
# mask = postprocessing.read_and_process_img(img_name, test_infer, block_size=2048, crop_size=2000)
mask = postprocessing.read_and_process_img(img_name, foo, block_size=2048, crop_size=2000)

In [ ]:
postprocessing._plot_img(mask.to('cpu'))

In [ ]:
mask01 = np.uint8(postprocessing.float_mask2_01uint8(mask).to('cpu'))

In [ ]:
dst_folder = Path('./output') / str(datetime.now()).replace(' ', '_')
dst_folder.mkdir(parents=True, exist_ok=True)
mask_name = dst_folder / img_name.name
mask_name

In [ ]:
utils.save_tiff_uint8_single_band(mask01, mask_name)

In [ ]:
fd, (h, w), channel = utils.get_basics_rasterio(mask_name)
readed_mask = utils.get_tiff_block(fd, 0, 0, w, h, 1)[0]
readed_mask.shape, readed_mask.dtype, readed_mask.max()

In [ ]:
postprocessing._plot_img(readed_mask)

In [ ]:
fd, shape, channel = utils.get_basics_rasterio(img_name)
h, w = shape
img = utils.get_tiff_block(fd, 0, 0, w, h, 3)
img.max(), img.shape, img.dtype

In [ ]:
merged_img = img.copy()
merged_img[1] = readed_mask*200

In [ ]:
postprocessing._plot_img(merged_img)

In [ ]:
postprocessing._plot_img(merged_img[:, 10000:12000, 25000:28000])

# Tests

In [ ]:
postprocessing.postprocess_test_folder(test_inf, src_folder, './output')